In [1]:
import numpy as np

save_file = 'data.npy'
with open(save_file, 'rb') as f:
    X = np.load(f)
    Y = np.load(f)

In [2]:
X = X.reshape(27000, -1)

In [3]:
Y.shape, X.shape

((27000,), (27000, 53248))

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((21600, 53248), (5400, 53248), (21600,), (5400,))

In [5]:
del X
del Y

# random search xgboost: Memory porblems + takes too much time

In [6]:
from xgboost import XGBClassifier

min_child_weight = [1, 5, 10]
gamma = [0.5, 1, 1.5, 2, 5]
subsample = [0.6, 0.8, 1.0]
colsample_bytree = [0.6, 0.8, 1.0]
max_depth = [3, 4, 5]

random_grid = {'min_child_weight': min_child_weight,
               'gamma': gamma,
               'subsample': subsample,
               'colsample_bytree': colsample_bytree,
               'max_depth': max_depth}

model = XGBClassifier(tree_method='gpu_hist', gpu_id=0, use_label_encoder=False, nthread=11, learning_rate=0.02, n_estimators=1000)

In [7]:
from sklearn.model_selection import RandomizedSearchCV

r_search = RandomizedSearchCV(model, param_distributions=random_grid, n_iter=100, scoring='roc_auc', n_jobs=4, cv=3, verbose=2, random_state=42)

In [ ]:
r_search.fit(X_train[:1000], Y_train[:1000])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [37]:
y_pred = model.predict(X_test)

# Alternative

In [6]:
from xgboost import XGBClassifier

params = {'learning_rate': 0.01,
          'max_depth': 3,
          'subsample': 0.8,
          'colsample_bytree': 1,
          'gamma': 1}

n_estimators = [50, 100, 150, 200, 250, 500, 1000]

In [ ]:
from sklearn import metrics

for num_est in n_estimators:
    clf = XGBClassifier(**params, n_estimators = num_est, nthread=11, use_label_encoder=False)
    clf.fit(X_train[:1000], Y_train[:1000])
    y_pred = clf.predict(X_test)
    print("n_estimators:{0} Accuracy:".format(num_est), metrics.accuracy_score(Y_test, y_pred))

[14:28:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [9]:
from sklearn import metrics

clf = XGBClassifier(n_estimators = 50, use_label_encoder=False)
clf.fit(X_train[:], Y_train[:])
y_pred = clf.predict(X_test)
print("Accuracy:", metrics.accuracy_score(Y_test, y_pred))

[14:46:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 